In [1]:
import keras  
from gensim.models import Word2Vec

Using TensorFlow backend.
D:\ANACONDA\envs\tensorflownt\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import numpy as np   
from keras.utils import plot_model,np_utils  
from keras.preprocessing.image import ImageDataGenerator  
from keras.models import *  
from keras.layers import *  
from keras import backend as K  
import h5py 
import random

# Load Data

In [3]:
filename = "cp.txt"  
raw_text = open(filename).read()  
raw_text = raw_text.lower()

## Chars to interger vectors: build rnn's input

In [4]:
chars = sorted(list(set(raw_text)))  
char_to_int = dict((c, i) for i, c in enumerate(chars))  
int_to_char = dict((i, c) for i, c in enumerate(chars)) 

In [5]:
print(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', ',', '-', '.', '0', '1', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '‘', '’', '…']


In [6]:
n_chars = len(raw_text)  
n_vocab = len(chars)

In [7]:
print('vocal: ',n_vocab)

vocal:  54


## cut the text in semi-redundant sequences of sqe_length characters

In [8]:
seq_length = 36  #how to decide
step = 3
dataX = []  
dataY = [] 

In [9]:
for i in range(0, n_chars - seq_length, step):  
    seq_in = raw_text[i:i + seq_length]  
    #seq_out = raw_text[i + seq_length]  
    seq_out = raw_text[i+1:i+1+seq_length] 
    dataX.append([char_to_int[char] for char in seq_in])  
    #dataY.append(char_to_int[seq_out])
    dataY.append([char_to_int[c] for c in seq_out])

In [10]:
print(dataX[:3])
print(dataY[:3])

[[43, 44, 33, 36, 36, 1, 37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1], [36, 36, 1, 37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30, 29, 29], [37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30, 29, 29, 36, 1, 37]]
[[44, 33, 36, 36, 1, 37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30], [36, 1, 37, 49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30, 29, 29, 36], [49, 1, 32, 29, 25, 42, 44, 1, 25, 38, 28, 1, 32, 39, 36, 28, 1, 37, 49, 1, 44, 39, 38, 31, 45, 29, 0, 33, 1, 30, 29, 29, 36, 1, 37, 49]]


In [11]:
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns) 

Total Patterns:  34404


## reshape X to be [samples, time steps, features] , and so do Y

In [12]:
 #X = np.reshape(dataX, (n_patterns, seq_length, 1))  
#normalize
#X = X / float(n_vocab)
#Y = np_utils.to_categorical(dataY) 
X = np.zeros((n_patterns, seq_length, n_vocab), dtype = np.bool)
Y = np.zeros((n_patterns, seq_length, n_vocab), dtype=np.bool)
#Y = np.zeros((n_patterns, n_vocab), dtype=np.bool)
for i, sentence in enumerate(dataX):
    for t, char in enumerate(sentence):
        X[i, t, char] = 1
    #Y[i, dataY[i]] = 1
for i, sentence in enumerate(dataY):
    for t, char in enumerate(sentence):
        Y[i, t, char] = 1    

In [13]:
#Set heckpoints and save weights
#filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"  
#checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')  
#callbacks_list = [checkpoint]  

# Build rnn Model

In [14]:
K.clear_session()

In [15]:
model = Sequential()  
#model.add(LSTM(128, input_shape=(X.shape[1], X.shape[2]), return_sequences=True, name='LSTM1'))  
#model.add(Dropout(0.2, name='Droupout1'))
#model.add(LSTM(128,input_shape=(X.shape[1], X.shape[2]),return_sequences=False,name='LSTM2')) 
#model.add(Dropout(0.2,name='Droupout2'))
#model.add(Flatten(name='Flatten'))  
#model.add(Dense(n_vocab,activation='softmax',name='output')) 

model.add(LSTM(128, input_dim=n_vocab,return_sequences=True))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))

optimizer = optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

model.summary()

D:\ANACONDA\envs\tensorflownt\lib\site-packages\ipykernel_launcher.py:9: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  if __name__ == '__main__':
D:\ANACONDA\envs\tensorflownt\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, input_shape=(None, 54), return_sequences=True)`
  if __name__ == '__main__':


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, None, 128)         93696     
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 128)         131584    
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 128)         0         
_________________________________________________________________
dense_1 (Dense)              (None, None, 54)          6966      
_________________________________________________________________
activation_1 (Activation)    (None, None, 54)          0         
Total params: 232,246
Trainable params: 232,246
Non-tra

In [16]:
#import pydot
#import graphviz
#plot_model(model, to_file='model.png') 

# Training the Model

In [17]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

nit = 1   # number of training iterations
nepoch_per_it = 20  # number of epochs per iterations
# Main iteration loop
for it in range(nit):
    # Continue the fit of the model
    init_epoch = it*nepoch_per_it
    model.fit(X, Y, epochs=nepoch_per_it, batch_size=128,verbose=0)#, callbacks=callbacks_list)  

    #Output generated text after each iteration
    print('-'*50)
    print("iter = ",it)
    start = random.randint(0, n_chars - seq_length -1)
    generated = ''
    sentence = raw_text[start:start+seq_length]
    generated +=sentence
    print('----- Generating with seed: "' + sentence + '"')
    for i in range(400):
        x_pred = np.zeros((1, seq_length, n_vocab))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_to_int[char]]=1
        preds = model.predict(x_pred, verbose=0)[0]
        next_int = sample(preds[len(sentence)-1])
        next_char = int_to_char[next_int]
        generated += next_char
        sentence = sentence[1:]+next_char
    print(generated)
           
model.save('lyrics_pre.h5')  

--------------------------------------------------
iter =  0
----- Generating with seed: " ooh ooh ooh ooh

you and me are flo"


D:\ANACONDA\envs\tensorflownt\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


 ooh ooh ooh ooh

you and me are floating on a tidal wave together
all my heart over that
you're the one i love to do i go to fall on me
tired you sliming ever cause you here beside me
but you mean more meant to know
it's no it is friends
i'll get a folling all last
i call it true
i call it magic

i saw speed living in perfect day

i don't know what houst set love
stop out of the day
the same stering in high
so on this hady nemers t


# Generate Lyrics

In [18]:
generated = ''
sentence = 'you and me'
generated +=sentence
print('----- Generating with seed: "' + sentence + '"')
print ('The generated text is:\n')
for i in range(400):
    x_pred = np.zeros((1, len(sentence), n_vocab))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_to_int[char]]=1
    preds = model.predict(x_pred, verbose=0)[0]
    next_int = sample(preds[len(sentence)-1])
    next_char = int_to_char[next_int]
    generated += next_char
    sentence = sentence[1:]+next_char
print(generated)

----- Generating with seed: "you and me"
The generated text is:



D:\ANACONDA\envs\tensorflownt\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


you and me
and they're it be saved, lettrees in the kind of pare
you don't then lie
to the seach coming home;

watch your heavy looking for gold
in the pope
why pood out ifeel never get hurt
and me your own politik

feeling the ripten watch is just a comy and sen
oh and i heard it on a roof, named everything's not gonna stand to go
don't believe my morting bout it loud, as mine

where do won't see the beaut
